# Красивая структура бд (1 балл)

### Сначала мы спроектировали БД для накопления инфомарции по взаимодействию с чат-ботом

![alt text](db.drawio.png "Database")

### Далее создали и запустил докер-контейнер PostgreSQL командой

```bash
docker run -p 5432:5432 --name hse_postgres -e POSTGRES_PASSWORD=sql -d postgres
```

`-p 5432:5432 `: маппинг порта докера и внешнего порта для соединения с контейнером  
`--name hse_postgres`: название контейнера  
`-e POSTGRES_PASSWORD=sql`: пароль  
`postgres`: имя образа  

### Затем создали таблицы и внесли свои тестовые данные

```sql
CREATE TABLE users (
    userid INTEGER PRIMARY KEY,
    name VARCHAR(255),
    type VARCHAR(255)
);

CREATE TABLE queries (
    queryid INTEGER PRIMARY KEY,
    message VARCHAR(255),
    validity VARCHAR(255),
    date_time TIMESTAMP,
    userid INTEGER,
    FOREIGN KEY (userid) REFERENCES users(userid)
);

CREATE TABLE responses (
    responseid INTEGER PRIMARY KEY, 
    text VARCHAR(255), 
    date_time TIMESTAMP,
    queryid INTEGER, 
    FOREIGN KEY (queryid) REFERENCES queries(queryid));
```

#  Скрипт на языке программирования (не SQL!) с обращением к БД (1 балл)

### Подключимся к БД через python, чтобы наглядно выполнить проектные задания 


In [ ]:
# установим базовую библиотеку для работы с postgres и визуализации
pip install psycopg2

In [1]:
# Подключаемся к БД 
import psycopg2

conn = psycopg2.connect(
    dbname='postgres',
    user='postgres',
    password='sql',
    host='localhost'
    )

conn.autocommit = True # поставлю True, чтобы не делать каждый раз commit, если буду что-то менять
cursor = conn.cursor() 

# Тестовые данные и запросы Антона

### Дополнительные тестовые данные
Для выполнения запросов я сгенерировал тестовые данные.

Запросы можно выполнять и в командной строке через psql, выполнив команду: 
```bash
docker exec -it hse_postgres psql -U postgres
```
Но мне удобнее использовать **PgAdmin 4**
```sql
INSERT INTO users (userid, name, type) VALUES
    (0, 'Anton', 'admin')
    (1, 'John', 'user'),
    (2, 'Maria', 'moderator'),
    (3, 'Anna', 'user'),
    (4, 'Peter', 'admin'),
    (5, 'Sophia', 'user'),
    (6, 'Michael', 'moderator'),
    (7, 'Emma', 'user'),
    (8, 'Alexander', 'admin'),
    (9, 'Olivia', 'moderator'),
    (10, 'William', 'user');

INSERT INTO queries (queryid, message, validity, date_time, userid) VALUES
    (0, 'test_text', 'valid', '2024-03-17 13:57:45', (select userid from users where name = 'Anton')),
    (1, 'Another test message', 'valid', '2024-03-17 14:30:00', 1),
    (2, 'Yet another test message', 'invalid', '2024-03-17 15:00:00', 2),
    (3, 'Query 3', 'valid', '2024-03-17 16:30:00', 3),
    (4, 'Query 4', 'invalid', '2024-03-17 17:00:00', 4),
    (5, 'Query 5', 'valid', '2024-03-17 18:00:00', 5),
    (6, 'Query 6', 'invalid', '2024-03-17 19:00:00', 6),
    (7, 'Query 7', 'valid', '2024-03-17 20:00:00', 7),
    (8, 'Query 8', 'invalid', '2024-03-17 21:00:00', 8),
    (9, 'Query 9', 'valid', '2024-03-17 22:00:00', 9),
    (10, 'Query 10', 'invalid', '2024-03-17 23:00:00', 10);

INSERT INTO responses (responseid, text, date_time, queryid) VALUES
    (0, 'test_response', '2024-03-17 13:57:45', (select queryid from queries where message = 'test_text'))
    (1, 'Another test response', '2024-03-17 14:35:00', 1),
    (2, 'Yet another test response', '2024-03-17 15:05:00', 2),
    (3, 'Response 3', '2024-03-17 16:35:00', 3),
    (4, 'Response 4', '2024-03-17 17:05:00', 4),
    (5, 'Response 5', '2024-03-17 18:05:00', 5),
    (6, 'Response 6', '2024-03-17 19:05:00', 6),
    (7, 'Response 7', '2024-03-17 20:05:00', 7),
    (8, 'Response 8', '2024-03-17 21:05:00', 8),
    (9, 'Response 9', '2024-03-17 22:05:00', 9),
    (10, 'Response 10', '2024-03-17 23:05:00', 10);

```

# Where с объединением трёх таблиц в where (без использования join!) (0.4 балла)

In [ ]:
sql = '''SELECT *
FROM users, queries, responses
WHERE users.userid = queries.userid
AND queries.queryid = responses.queryid
AND users.name = 'Anton';
 '''
  
cursor.execute(sql)

for table in cursor.fetchall():
    print(table)

(0, 'Anton', 'admin', 0, 'test_text', 'valid', datetime.datetime(2024, 3, 17, 13, 57, 45), 0, 0, 'test_response', datetime.datetime(2024, 3, 17, 13, 57, 45), 0)


# Вложенный select (1 балл):

In [ ]:
# Пример вложенного запроса:

sql = '''SELECT *
FROM users
WHERE userid IN (SELECT userid FROM queries WHERE validity = 'valid');
'''
  
cursor.execute(sql)

for table in cursor.fetchall():
    print(table)

(9, 'Olivia', 'moderator')
(7, 'Emma', 'user')
(0, 'Anton', 'admin')
(1, 'John', 'user')
(3, 'Anna', 'user')
(5, 'Sophia', 'user')


In [ ]:
# Пример запроса, который нельзя переписать без вложенного запроса:
sql = '''SELECT *
FROM users
WHERE name = (SELECT name FROM users WHERE userid = 1);
'''
  
cursor.execute(sql)

for table in cursor.fetchall():
    print(table)

(1, 'John', 'user')


In [ ]:
# Пример вложенного запроса с использованием WITH:
sql = '''WITH user_queries AS (
    SELECT userid, COUNT(*) AS num_queries
    FROM queries
    GROUP BY userid
)
SELECT u.userid, u.name, u.type, uq.num_queries
FROM users u
JOIN user_queries uq ON u.userid = uq.userid;;
'''
  
cursor.execute(sql)

for table in cursor.fetchall():
    print(table)

(0, 'Anton', 'admin', 1)
(1, 'John', 'user', 1)
(2, 'Maria', 'moderator', 1)
(3, 'Anna', 'user', 1)
(4, 'Peter', 'admin', 1)
(5, 'Sophia', 'user', 1)
(6, 'Michael', 'moderator', 1)
(7, 'Emma', 'user', 1)
(8, 'Alexander', 'admin', 1)
(9, 'Olivia', 'moderator', 1)
(10, 'William', 'user', 1)


# Join (1 балл)

In [ ]:
sql = '''SELECT *
FROM users
LEFT JOIN queries ON users.userid = queries.userid;
'''
  
cursor.execute(sql)

for table in cursor.fetchall():
    print(table)

(0, 'Anton', 'admin', 0, 'test_text', 'valid', datetime.datetime(2024, 3, 17, 13, 57, 45), 0)
(1, 'John', 'user', 1, 'Another test message', 'valid', datetime.datetime(2024, 3, 17, 14, 30), 1)
(2, 'Maria', 'moderator', 2, 'Yet another test message', 'invalid', datetime.datetime(2024, 3, 17, 15, 0), 2)
(3, 'Anna', 'user', 3, 'Query 3', 'valid', datetime.datetime(2024, 3, 17, 16, 30), 3)
(4, 'Peter', 'admin', 4, 'Query 4', 'invalid', datetime.datetime(2024, 3, 17, 17, 0), 4)
(5, 'Sophia', 'user', 5, 'Query 5', 'valid', datetime.datetime(2024, 3, 17, 18, 0), 5)
(6, 'Michael', 'moderator', 6, 'Query 6', 'invalid', datetime.datetime(2024, 3, 17, 19, 0), 6)
(7, 'Emma', 'user', 7, 'Query 7', 'valid', datetime.datetime(2024, 3, 17, 20, 0), 7)
(8, 'Alexander', 'admin', 8, 'Query 8', 'invalid', datetime.datetime(2024, 3, 17, 21, 0), 8)
(9, 'Olivia', 'moderator', 9, 'Query 9', 'valid', datetime.datetime(2024, 3, 17, 22, 0), 9)
(10, 'William', 'user', 10, 'Query 10', 'invalid', datetime.datetime(2

# Агрегация: group by, order by, аггрегация в результирующих полях (0.5):

In [ ]:
sql = '''SELECT userid, COUNT(*) AS num_queries
FROM queries
GROUP BY userid
ORDER BY num_queries DESC;
'''
  
cursor.execute(sql)

for table in cursor.fetchall():
    print(table)

(4, 1)
(0, 1)
(10, 1)
(6, 1)
(2, 1)
(9, 1)
(7, 1)
(3, 1)
(1, 1)
(5, 1)
(8, 1)


# Оконные функции (0.5 балла)

In [ ]:
sql = '''SELECT *,
       ROW_NUMBER() OVER(ORDER BY userid) AS row_num
FROM users;
'''
  
cursor.execute(sql)

for table in cursor.fetchall():
    print(table)

(0, 'Anton', 'admin', 1)
(1, 'John', 'user', 2)
(2, 'Maria', 'moderator', 3)
(3, 'Anna', 'user', 4)
(4, 'Peter', 'admin', 5)
(5, 'Sophia', 'user', 6)
(6, 'Michael', 'moderator', 7)
(7, 'Emma', 'user', 8)
(8, 'Alexander', 'admin', 9)
(9, 'Olivia', 'moderator', 10)
(10, 'William', 'user', 11)


# Тестовые данные и запросы Полины

### Дополнительные тестовые данные
Для выполнения запросов я сгенерировала тестовые данные
```sql
INSERT INTO users (userid, name, type) VALUES
    (0, 'SnowflakeAnton92', 'admin'),
    (1, 'IcyIvan88', 'user'),
    (2, 'MariyaMagic22', 'moderator'),
    (3, 'AnnaAurora37', 'user'),
    (4, 'PetrPolarBear55', 'admin'),
    (5, 'SofiyaSnowstorm19', 'user'),
    (6, 'MikhailMoonlight81', 'moderator'),
    (7, 'YekaterinaFrostbite26', 'user'),
    (8, 'AleksandrArcticFox10', 'admin'),
    (9, 'OlgaBlizzard73', 'moderator'),
    (10, 'VladimirIceberg47', 'user');


INSERT INTO queries (queryid, message, validity, date_time, userid) VALUES
    (0, 'Is two plus two equal to four?', 'valid', '2024-03-12 08:21:34', (select userid from users where name = 'SnowflakeAnton92')),
    (1, 'Can you tell me about quantum physics?', 'valid', '2024-03-13 10:45:22', 1),
    (2, 'Translate "hello" to French', 'invalid', '2024-03-14 16:02:58', 2),
    (3, 'How does a neural network work?', 'valid', '2024-03-15 20:30:10', 3),
    (4, 'Explain the concept of time dilation', 'invalid', '2024-03-16 11:50:45', 4),
    (5, 'What are the symptoms of COVID-19?', 'valid', '2024-03-17 14:15:29', 5),
    (6, 'Describe the process of photosynthesis', 'invalid', '2024-03-18 09:28:17', 6),
    (7, 'Can you summarize "War and Peace" by Leo Tolstoy?', 'valid', '2024-03-19 22:42:53', 7),
    (8, 'What are the main features of the Python programming language?', 'invalid', '2024-03-20 18:55:41', 8),
    (9, 'Explain the concept of black holes in astrophysics', 'valid', '2024-03-21 14:09:27', 9),
    (10, 'What is the meaning of life?', 'invalid', '2024-03-22 19:23:14', 10),
    (11, 'What is the capital of Australia?','valid','2024-03-22 20:23:14', 10),
    (12, 'Who wrote the famous novel "To Kill a Mockingbird"?','valid','2024-03-22 21:23:14', 10),
    (13, 'What is the chemical symbol for gold?','valid','2024-03-22 22:23:14',10),
    (14, 'What year did the Titanic sink?','valid','2024-03-22 23:23:14',10),
    (15, 'What is the tallest mountain in the world?','valid','2024-03-23 00:23:14',10);
    


INSERT INTO responses (responseid, text, date_time, queryid) VALUES
    (0, 'Yes', '2024-03-17 13:57:45', (select queryid from queries where message = 'Is two plus two equal to four?'))
    (1, 'Quantum physics is a branch of physics that explores the behavior of particles at the smallest scales.', '2024-03-17 14:35:00', 1),
    (2, 'Bonjour', '2024-03-17 15:05:00', 2),
    (3, 'A neural network is a computational model inspired by the structure and functioning of biological neural networks.', '2024-03-17 16:35:00', 3),
    (4, 'Time dilation is a phenomenon predicted by Einsteins theory of relativity.', '2024-03-17 17:05:00', 4),
    (5, 'The symptoms of COVID-19 include fever, cough, shortness of breath.', '2024-03-17 18:05:00', 5),
    (6, 'Photosynthesis is the process by which green plants convert light energy into chemical energy.', '2024-03-17 19:05:00', 6),
    (7, 'War and Peace is a novel by the Russian author Leo Tolstoy, which tells the story of Russian society during the Napoleonic Wars.', '2024-03-17 20:05:00', 7),
    (8, 'The main features of the Python programming language include its simplicity, readability, versatility, and extensive standard library.', '2024-03-17 21:05:00', 8),
    (9, 'Black holes are regions of spacetime where gravity is so strong that nothing, not even light, can escape from them.', '2024-03-17 22:05:00', 9),
    (10, 'The meaning of life is a philosophical question that has been pondered by humans for centuries, with various interpretations and perspectives.', '2024-03-17 23:05:00', 10)
    (11, 'Canberra', '2024-03-22 20:23:14', 11),
    (12, 'Harper Lee', '2024-03-22 21:23:14', 12),
    (13, 'Au', '2024-03-22 22:23:14', 13),
    (14, '1912', '2024-03-22 23:23:14', 14),
    (15, 'Mount Everest', '2024-03-23 00:23:14', 15);

```

# Where с объединением трёх таблиц в where (без использования join!) (0.4 балла)

In [24]:
#Выведем всю информацию по запросам пользователей с типом "user"
sql = '''SELECT 
    users.userid,
    users.name AS username,
    users.type AS usertype,
    queries.queryid,
    queries.message AS query_message,
    queries.validity AS query_validity,
    queries.date_time AS query_datetime,
    responses.responseid,
    responses.text AS response_text,
    responses.date_time AS response_datetime
FROM 
    users,
    queries,
    responses
WHERE 
    queries.userid = users.userid
    AND responses.queryid = queries.queryid
	AND users.type = 'user';
 '''
  
cursor.execute(sql)

for table in cursor.fetchall():
    print(table)

(1, 'IcyIvan88', 'user', 1, 'Can you tell me about quantum physics?', 'valid', datetime.datetime(2024, 3, 13, 10, 45, 22), 1, 'Quantum physics is a branch of physics that explores the behavior of particles at the smallest scales.', datetime.datetime(2024, 3, 17, 14, 35))
(3, 'AnnaAurora37', 'user', 3, 'How does a neural network work?', 'valid', datetime.datetime(2024, 3, 15, 20, 30, 10), 3, 'A neural network is a computational model inspired by the structure and functioning of biological neural networks.', datetime.datetime(2024, 3, 17, 16, 35))
(5, 'SofiyaSnowstorm19', 'user', 5, 'What are the symptoms of COVID-19?', 'valid', datetime.datetime(2024, 3, 17, 14, 15, 29), 5, 'The symptoms of COVID-19 include fever, cough, shortness of breath.', datetime.datetime(2024, 3, 17, 18, 5))
(7, 'YekaterinaFrostbite26', 'user', 7, 'Can you summarize "War and Peace" by Leo Tolstoy?', 'valid', datetime.datetime(2024, 3, 19, 22, 42, 53), 7, 'War and Peace is a novel by the Russian author Leo Tolstoy

# Вложенный select (1 балл):

In [25]:
# Выведем всех пользователей, отправивших запросы после 2024-03-17 18:00:00

sql = '''SELECT name, type
FROM users
WHERE userid IN (SELECT userid FROM queries WHERE date_time > '2024-03-17 18:00:00');

'''
  
cursor.execute(sql)

for table in cursor.fetchall():
    print(table)

('MikhailMoonlight81', 'moderator')
('YekaterinaFrostbite26', 'user')
('AleksandrArcticFox10', 'admin')
('OlgaBlizzard73', 'moderator')
('VladimirIceberg47', 'user')


In [26]:
# Выведем сколько запросов у каждого пользователя
sql = '''SELECT 
    name,
    type,
    (SELECT COUNT(*) FROM queries WHERE userid = users.userid) AS total_queries
FROM users;

'''
  
cursor.execute(sql)

for table in cursor.fetchall():
    print(table)

('SnowflakeAnton92', 'admin', 1)
('IcyIvan88', 'user', 1)
('MariyaMagic22', 'moderator', 1)
('AnnaAurora37', 'user', 1)
('PetrPolarBear55', 'admin', 1)
('SofiyaSnowstorm19', 'user', 1)
('MikhailMoonlight81', 'moderator', 1)
('YekaterinaFrostbite26', 'user', 1)
('AleksandrArcticFox10', 'admin', 1)
('OlgaBlizzard73', 'moderator', 1)
('VladimirIceberg47', 'user', 6)


In [27]:
# Пример вложенного запроса с использованием WITH:
# Выведем запрос и ответ на него
sql = '''WITH UserQueries AS (
    SELECT q.message AS query_message, r.text AS response_text
    FROM queries q
    INNER JOIN responses r ON q.queryid = r.queryid
)
SELECT *
FROM UserQueries;

'''
  
cursor.execute(sql)

for table in cursor.fetchall():
    print(table)

('Is two plus two equal to four?', 'Yes')
('Can you tell me about quantum physics?', 'Quantum physics is a branch of physics that explores the behavior of particles at the smallest scales.')
('Translate "hello" to French', 'Bonjour')
('How does a neural network work?', 'A neural network is a computational model inspired by the structure and functioning of biological neural networks.')
('Explain the concept of time dilation', 'Time dilation is a phenomenon predicted by Einsteins theory of relativity.')
('What are the symptoms of COVID-19?', 'The symptoms of COVID-19 include fever, cough, shortness of breath.')
('Describe the process of photosynthesis', 'Photosynthesis is the process by which green plants convert light energy into chemical energy.')
('Can you summarize "War and Peace" by Leo Tolstoy?', 'War and Peace is a novel by the Russian author Leo Tolstoy, which tells the story of Russian society during the Napoleonic Wars.')
('What are the main features of the Python programming l

# Join (1 балл)

In [50]:
sql = '''SELECT u.name AS username, q.message AS query_message, r.text AS response_text
FROM users u
JOIN queries q ON u.userid = q.userid
JOIN responses r ON q.queryid = r.queryid
WHERE q.validity = 'valid';
'''
  
cursor.execute(sql)

for table in cursor.fetchall():
    print(table)

('SnowflakeAnton92', 'Is two plus two equal to four?', 'Yes')
('IcyIvan88', 'Can you tell me about quantum physics?', 'Quantum physics is a branch of physics that explores the behavior of particles at the smallest scales.')
('AnnaAurora37', 'How does a neural network work?', 'A neural network is a computational model inspired by the structure and functioning of biological neural networks.')
('SofiyaSnowstorm19', 'What are the symptoms of COVID-19?', 'The symptoms of COVID-19 include fever, cough, shortness of breath.')
('YekaterinaFrostbite26', 'Can you summarize "War and Peace" by Leo Tolstoy?', 'War and Peace is a novel by the Russian author Leo Tolstoy, which tells the story of Russian society during the Napoleonic Wars.')
('OlgaBlizzard73', 'Explain the concept of black holes in astrophysics', 'Black holes are regions of spacetime where gravity is so strong that nothing, not even light, can escape from them.')
('VladimirIceberg47', 'What is the capital of Australia?', 'Canberra')
(

# Агрегация: group by, order by, аггрегация в результирующих полях (0.5):

In [52]:
#Запрос, который показывает пользователя, который пишет самые длинные запросы (расчет по средней длине строки-запроса)

sql = '''SELECT u.name AS user_name, AVG(CHAR_LENGTH(q.message)) AS avg_query_length
FROM users u
JOIN queries q ON u.userid = q.userid
GROUP BY u.userid, u.name
ORDER BY avg_query_length DESC
LIMIT 1;
'''
  
cursor.execute(sql)

for table in cursor.fetchall():
    print(table)

('AleksandrArcticFox10', Decimal('62.0000000000000000'))


In [51]:
# Выведем пользователей, которые отправили > 1 запроса
sql = '''SELECT u.userid, u.name, COUNT(q.queryid) 
FROM users u
INNER JOIN queries q ON u.userid = q.userid
GROUP BY u.userid, u.name
HAVING COUNT(q.queryid) > 1;
'''

cursor.execute(sql)

for table in cursor.fetchall():
    print(table)

(10, 'VladimirIceberg47', 6)


# Оконные функции (0.5 балла)

In [49]:
#запрос показывает, сколько запросов с типом valid у каждого пользователя
sql = '''WITH UserValidQueryCounts AS (
    SELECT u.userid, u.name, q.validity,
           COUNT(*) OVER (PARTITION BY u.userid) AS total_queries,
           COUNT(*) FILTER (WHERE q.validity = 'valid') OVER (PARTITION BY u.userid) AS valid_queries_count
    FROM users u
    LEFT JOIN queries q ON u.userid = q.userid
)
SELECT DISTINCT userid, name, valid_queries_count
FROM UserValidQueryCounts
WHERE valid_queries_count > 0
ORDER BY userid;
'''
  
cursor.execute(sql)

for table in cursor.fetchall():
    print(table)

(0, 'SnowflakeAnton92', 1)
(1, 'IcyIvan88', 1)
(3, 'AnnaAurora37', 1)
(5, 'SofiyaSnowstorm19', 1)
(7, 'YekaterinaFrostbite26', 1)
(9, 'OlgaBlizzard73', 1)
(10, 'VladimirIceberg47', 5)


# Мы создали docker-compose и файл, инициализирующий БД init.sql

# Docker (1 балл)

Создан файл docker-compose.yaml

```yaml
version: '3.9'

services:
  db:
    image: postgres
    restart: always
    environment:
      POSTGRES_PASSWORD: sql
    ports:
      - "5432:5432"
    volumes:
      - ./init.sql:/docker-entrypoint-initdb.d/init.sql
```
`init.sql` - необходим для того, чтобы задать код для инициализации базы данных (код указан в самом начале: создание таблиц и наполнение их тестовыми данными)

Поместить этот файл в ту же директорию, где находится docker-compose.yml. Затем выполнить команду docker-compose up -d, чтобы запустить контейнер

# Связанное решение в серверной sql-базе данных (1 балл), Знания, выходящие за рамки курса - 0.6 балла
Почему был выбран Postgres? Нам понравился тот факт, что для данной БД есть удобные асинхронные библиотеки, чтобы делать производительные сервисы, поэтому было принято решение: попробовать реализовать чат-бот, используя стек: `fastapi` + `asyncpg`
Мы немного изменили БД для создания api и вставки данных в БД, т.к. в "игрушечной" версии id для полей мы прописывали сами. В сервисной реализации это будут автоинкременты BIGSERIAL. Более подробно в init.sql

Установим asyncpg. Можно было бы использовать SQLAlchemy, но это ORM со специфическим синтаксисом, поэтому решили попробовать asyncpg

In [7]:
pip install asyncpg asyncio fastapi uvicorn

     |████████████████████████████████| 101 kB 1.6 MB/s ta 0:00:01
You should consider upgrading via the '/Users/anper/Documents/DB/MySQL/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


Далее код в main.py